# Financial Sentiment Analysis (Single)

In this program, I run a sentiment analysis of a single company based on financial news articles.

The company that I am targeting is Nvidia [NVDA]  

From that sentiment, I will use the data to try and predict the price movement of that company's stock price.  

## Fetching News Articles

The first step is to fetch the news articles.  

I am using `NewsAPI` to get articles quickly and easily. Then, I use `pandas` to put the articles into a dataframe, where I can collect and read the data easier.  

**Filtering articles:**  
Filter articles that only exist  
- `NewsAPI` sometimes fetches articles that were removed  

**Extracting the data:**  
Extract only the necessary data from the articles
- Title
- Content
- Publish Date  

All others can be discarded.  

Both of these steps are part of the cleaning data step that is next in text preprocessing.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
# get path to the environment file
env_path = '../../config/.env'
load_dotenv(env_path)

True

In [3]:
# import datetime, timedelta modules from datetime
from datetime import datetime, timedelta

In [4]:
# import newsapi package
from newsapi import NewsApiClient

In [5]:
# init newsapi
newsapi = NewsApiClient(api_key=os.getenv('NEWS_API_KEY'))

In [6]:
company = "Nvidia"
days_back = 29
end_date = datetime.now()
start_date = end_date - timedelta(days=days_back)

In [7]:
# fetch all articles that mention Nvidia
all_articles = newsapi.get_everything(q=company,
                                      from_param=start_date.strftime('%Y-%m-%d'),
                                      to=end_date.strftime('%Y-%m-%d'),
                                      language='en')

In [8]:
import pandas as pd
pd.__version__

'2.2.3'

In [9]:
# place all_articles into a dataframe
all_articles_df = pd.DataFrame(all_articles['articles'])
all_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Jeremy Gan,ByteDance will reportedly use Huawei chips to ...,"As first reported by Reuters, ByteDance, the C...",https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-30T15:48:46Z,"If you click 'Accept all', we and our partners..."
3,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia's days of absolute dominance in AI coul...,Startups like SambaNova and Cerebras are chall...,https://www.businessinsider.com/nvidia-competi...,https://i.insider.com/66f425a5cfb7f307e574cbbe...,2024-09-26T09:00:02Z,"Rodrigo Liang, CEO and co-founder of SambaNova..."
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Quartz India'}",Rocio Fabbro,Nvidia is back in the $3 trillion club — and J...,Nvidia (NVDA) chief executive Jensen Huang is ...,https://qz.com/nvidia-huang-stock-market-cap-3...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-09-26T12:25:00Z,In This Story\r\nNvidia (NVDA) chief executive...
96,"{'id': None, 'name': 'PC Gamer'}",Jeremy Laird,Sony confirms PS5 Pro has next-gen ray-tracing...,"Sony says it ""motivated"" AMD to create better ...",https://www.pcgamer.com/hardware/sony-confirms...,https://s.yimg.com/ny/api/res/1.2/nR_ruZqbYtL3...,2024-09-13T10:29:30Z,"When you buy through links on our articles, Fu..."
97,"{'id': None, 'name': 'Windows Central'}",c.cale.hunt@gmail.com (Cale Hunt),Acer's formidable new Predator Orion 7000 desk...,Acer's Nitro and Predator gaming lineups are g...,https://www.windowscentral.com/hardware/laptop...,https://cdn.mos.cms.futurecdn.net/YsWSvvNbuwKe...,2024-09-04T13:00:58Z,What you need to know\r\n<ul><li>Acer announce...
98,"{'id': 'polygon', 'name': 'Polygon'}",Cameron Faulkner,LG’s entry-level OLED is a great gaming TV — a...,If you’ve been curious about buying an OLED TV...,https://www.polygon.com/good-deals/453424/lg-b...,https://platform.polygon.com/wp-content/upload...,2024-09-19T14:20:58Z,If you’ve been curious about buying an OLED TV...


In [10]:
# filter articles function
# only filters valid articles
# valid meaning: article exists and description of article exists
def filter_removed_articles(articles):
    return [article for article in articles if article.get('title') != '[Removed]']

In [11]:
# filter the all_articles
valid_articles = filter_removed_articles(all_articles['articles'])

In [12]:
valid_articles_df = pd.DataFrame(valid_articles)
valid_articles_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Lawrence Bonk,DOJ subpoenas NVIDIA as part of antitrust prob...,The DOJ has sent subpoenas to NVIDIA and other...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-04T15:34:35Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Gizmodo.com'}",Kyle Barr,The Leaked Nvidia RTX 5090 Has So Many Cores I...,Get ready to watch the lights on your block di...,https://gizmodo.com/the-leaked-nvidia-rtx-5090...,https://gizmodo.com/app/uploads/2024/09/Nvidia...,2024-09-27T13:35:22Z,The GeForce RTX 4090 is already so big that an...
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Jeremy Gan,ByteDance will reportedly use Huawei chips to ...,"As first reported by Reuters, ByteDance, the C...",https://consent.yahoo.com/v2/collectConsent?se...,None,2024-09-30T15:48:46Z,"If you click 'Accept all', we and our partners..."
3,"{'id': 'business-insider', 'name': 'Business I...",Eugene Kim,This chart shows one potential advantage AWS's...,"AI chip investments by Amazon, Google, and Mic...",https://www.businessinsider.com/aws-ai-chips-w...,https://i.insider.com/6622c44b23b29110d3011ce1...,2024-09-26T09:00:02Z,Noah Berger/Getty Images\r\n<ul><li>Big tech c...
4,"{'id': 'business-insider', 'name': 'Business I...",Emma Cosgrove,Nvidia's days of absolute dominance in AI coul...,Startups like SambaNova and Cerebras are chall...,https://www.businessinsider.com/nvidia-competi...,https://i.insider.com/66f425a5cfb7f307e574cbbe...,2024-09-26T09:00:02Z,"Rodrigo Liang, CEO and co-founder of SambaNova..."
...,...,...,...,...,...,...,...,...
94,"{'id': None, 'name': 'Quartz India'}",Rocio Fabbro,Nvidia is back in the $3 trillion club — and J...,Nvidia (NVDA) chief executive Jensen Huang is ...,https://qz.com/nvidia-huang-stock-market-cap-3...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-09-26T12:25:00Z,In This Story\r\nNvidia (NVDA) chief executive...
95,"{'id': None, 'name': 'PC Gamer'}",Jeremy Laird,Sony confirms PS5 Pro has next-gen ray-tracing...,"Sony says it ""motivated"" AMD to create better ...",https://www.pcgamer.com/hardware/sony-confirms...,https://s.yimg.com/ny/api/res/1.2/nR_ruZqbYtL3...,2024-09-13T10:29:30Z,"When you buy through links on our articles, Fu..."
96,"{'id': None, 'name': 'Windows Central'}",c.cale.hunt@gmail.com (Cale Hunt),Acer's formidable new Predator Orion 7000 desk...,Acer's Nitro and Predator gaming lineups are g...,https://www.windowscentral.com/hardware/laptop...,https://cdn.mos.cms.futurecdn.net/YsWSvvNbuwKe...,2024-09-04T13:00:58Z,What you need to know\r\n<ul><li>Acer announce...
97,"{'id': 'polygon', 'name': 'Polygon'}",Cameron Faulkner,LG’s entry-level OLED is a great gaming TV — a...,If you’ve been curious about buying an OLED TV...,https://www.polygon.com/good-deals/453424/lg-b...,https://platform.polygon.com/wp-content/upload...,2024-09-19T14:20:58Z,If you’ve been curious about buying an OLED TV...


In [13]:
# extract article essentials function
# extract only the title, description, and content from the articles
def extract_article_essentials(articles):
    return [{'title': article['title'], 'content': article['content'], 'publishedAt': article['publishedAt']} for article in articles]

In [14]:
extracted_articles = extract_article_essentials(valid_articles)

In [15]:
extracted_articles_df = pd.DataFrame(extracted_articles)
extracted_articles_df

,title,content,publishedAt
0,DOJ subpoenas NVIDIA as part of antitrust prob...,"If you click 'Accept all', we and our partners...",2024-09-04T15:34:35Z
1,The Leaked Nvidia RTX 5090 Has So Many Cores I...,The GeForce RTX 4090 is already so big that an...,2024-09-27T13:35:22Z
2,ByteDance will reportedly use Huawei chips to ...,"If you click 'Accept all', we and our partners...",2024-09-30T15:48:46Z
3,This chart shows one potential advantage AWS's...,Noah Berger/Getty Images\r\n<ul><li>Big tech c...,2024-09-26T09:00:02Z
4,Nvidia's days of absolute dominance in AI coul...,"Rodrigo Liang, CEO and co-founder of SambaNova...",2024-09-26T09:00:02Z
...,...,...,...
94,Nvidia is back in the $3 trillion club — and J...,In This Story\r\nNvidia (NVDA) chief executive...,2024-09-26T12:25:00Z
95,Sony confirms PS5 Pro has next-gen ray-tracing...,"When you buy through links on our articles, Fu...",2024-09-13T10:29:30Z
96,Acer's formidable new Predator Orion 7000 desk...,What you need to know\r\n<ul><li>Acer announce...,2024-09-04T13:00:58Z
97,LG’s entry-level OLED is a great gaming TV — a...,If you’ve been curious about buying an OLED TV...,2024-09-19T14:20:58Z


In [16]:
extracted_articles_df.to_csv(os.path.join('../DataFrames', 'extracted_articles_df.tsv'), index=False)